<a href="https://colab.research.google.com/github/Ngum12/PDF-Mov_conv/blob/main/Ghana_crop_competition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/ghana-crop-disease-detection-challenge20241003-7123-59ht2i

/content/drive/MyDrive/ghana-crop-disease-detection-challenge20241003-7123-59ht2i


In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 870.5/870.5 kB 45.7 MB/s eta 0:00:00


In [ ]:
# First, mount Google Drive and set up the environment
from google.colab import drive
drive.mount('/content/drive')

# Import required libraries
import os
import numpy as np
import pandas as pd
import cv2
import yaml
import shutil
import multiprocessing
from pathlib import Path
from PIL import Image
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from ultralytics import YOLO

# Install required packages
!pip install -q ultralytics

# Configuration
class Config:
    """Global configuration settings"""
    SEED = 42
    IMAGE_SIZE = 1024
    BATCH_SIZE = 8
    EPOCHS = 10
    TRAIN_TEST_SPLIT = 0.25
    DEVICE = "cpu"  # Use "cpu" if GPU is not available
    MODEL_TYPE = 'yolov8n.pt'  # Base model architecture

# Define the base path to your project directory in Google Drive
BASE_PATH = '/content/drive/MyDrive/ghana-crop-disease-detection-challenge20241003-7123-59ht2i'

# Directory Structure Setup
class DirectoryManager:
    """Manages directory structure for the project"""
    def __init__(self, base_path: str):
        self.base_path = Path(base_path)
        self.dataset_dir = Path('/content/datasets/dataset')  # Local working directory
        self.setup_directories()

    def setup_directories(self):
        """Creates necessary directory structure"""
        # Define directory structure
        self.dirs = {
            'images': {
                'train': self.dataset_dir / 'images/train',
                'val': self.dataset_dir / 'images/val',
                'test': self.dataset_dir / 'images/test'
            },
            'labels': {
                'train': self.dataset_dir / 'labels/train',
                'val': self.dataset_dir / 'labels/val',
                'test': self.dataset_dir / 'labels/test'
            }
        }

        # Create directories
        for category in self.dirs.values():
            for dir_path in category.values():
                if dir_path.exists():
                    shutil.rmtree(str(dir_path))
                dir_path.mkdir(parents=True, exist_ok=True)

# Data Preprocessing
class DataPreprocessor:
    """Handles data preprocessing and preparation"""
    def __init__(self, base_path: Path):
        self.base_path = base_path

    def load_data(self):
        """Load and prepare training and testing data"""
        try:
            # Load CSV files with explicit path
            self.train_df = pd.read_csv(self.base_path / 'Train.csv')
            self.test_df = pd.read_csv(self.base_path / 'Test.csv')

            # Unzip images if needed
            images_zip_path = self.base_path / 'images.zip'
            if images_zip_path.exists():
                !unzip -q "{str(images_zip_path)}" -d "/content/images"

            # Add image paths
            self.train_df['image_path'] = [
                Path('/content/images/' + x) for x in self.train_df.Image_ID
            ]
            self.test_df['image_path'] = [
                Path('/content/images/' + x) for x in self.test_df.Image_ID
            ]

            # Create class mappings
            self.create_class_mappings()
            print("Data loaded successfully!")

        except Exception as e:
            print(f"Error loading data: {e}")
            raise

    def create_class_mappings(self):
        """Create mappings between class names and indices"""
        unique_classes = sorted(self.train_df['class'].unique().tolist())
        self.class_mapper = {
            x: y for x, y in zip(unique_classes, range(len(unique_classes)))
        }
        self.train_df['class_id'] = self.train_df['class'].map(self.class_mapper)

# Main execution
def main():
    """Main execution function"""
    try:
        # Initialize configuration
        config = Config()

        # Print current working directory and available files
        print("Current working directory:", os.getcwd())
        print("\nFiles in base directory:")
        !ls "{BASE_PATH}"

        # Initialize directory manager
        print("\nSetting up directory structure...")
        dir_manager = DirectoryManager(BASE_PATH)

        # Initialize data preprocessor
        print("\nLoading and preprocessing data...")
        preprocessor = DataPreprocessor(Path(BASE_PATH))
        preprocessor.load_data()

        # Continue with the rest of the pipeline...
        print("\nSetup completed successfully!")

        return preprocessor, dir_manager

    except Exception as e:
        print(f"Error in main execution: {e}")
        raise

if __name__ == "__main__":
    preprocessor, dir_manager = main()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Current working directory: /content/drive/My Drive/ghana-crop-disease-detection-challenge20241003-7123-59ht2i

Files in base directory:
dataset								     runs
data.yaml							     SampleSubmission.csv
ghana-crop-disease-detection-challenge20241003-7123-59ht2i	     Test.csv
images.zip							     Train.csv
manifest-8517f262e2ca61a7f7ac8fc1d32eb0be20241003-7123-1lvxb66.json  yolov8n.pt
Rail_Challenge_Starter.ipynb

Setting up directory structure...

Loading and preprocessing data...
Data loaded successfully!

Setup completed successfully!


In [ ]:
class DataPreprocessor:
    """
    Data Preprocessor for Crop Disease Detection
    Handles data loading, preprocessing, and augmentation
    """
    def __init__(self):
        """
        Initialize the DataPreprocessor with necessary attributes
        """
        # Paths
        self.base_path = Path('/content/drive/MyDrive/ghana-crop-disease-detection-challenge20241003-7123-59ht2i')
        self.dataset_path = Path('/content/dataset')

        # DataFrames
        self.train_df = None
        self.test_df = None
        self.val_df = None

        # Class mapping
        self.class_mapper = {}
        self.num_classes = 0

        # Statistics
        self.class_distribution = None
        self.image_dimensions = []

        # Create necessary directories
        self._create_directories()

    def _create_directories(self):
        """Create necessary directories for data processing"""
        directories = [
            self.dataset_path / 'images' / x for x in ['train', 'val', 'test']
        ] + [
            self.dataset_path / 'labels' / x for x in ['train', 'val', 'test']
        ]

        for dir_path in directories:
            dir_path.mkdir(parents=True, exist_ok=True)
            print(f"Created directory: {dir_path}")

    def load_data(self):
        """
        Load and prepare the dataset
        """
        try:
            print("Loading dataset...")

            # Load CSV files
            self.train_df = pd.read_csv(self.base_path / 'Train.csv')
            self.test_df = pd.read_csv(self.base_path / 'Test.csv')

            # Unzip images if needed
            images_zip_path = self.base_path / 'images.zip'
            if images_zip_path.exists():
                print("Extracting images...")
                !unzip -q "{str(images_zip_path)}" -d "/content/images"

            # Add image paths
            self.train_df['image_path'] = [
                Path('/content/images/' + x) for x in self.train_df.Image_ID
            ]
            self.test_df['image_path'] = [
                Path('/content/images/' + x) for x in self.test_df.Image_ID
            ]

            # Create class mappings
            self.create_class_mappings()

            # Analyze dataset
            self._analyze_dataset()

            print("Data loaded successfully!")
            self.print_dataset_info()

        except Exception as e:
            print(f"Error loading data: {e}")
            raise

    def create_class_mappings(self):
        """
        Create mappings between class names and indices
        """
        try:
            # Get unique classes and sort them
            unique_classes = sorted(self.train_df['class'].unique().tolist())

            # Create mapping dictionary
            self.class_mapper = {
                class_name: idx for idx, class_name in enumerate(unique_classes)
            }
            self.num_classes = len(self.class_mapper)

            # Add class IDs to training data
            self.train_df['class_id'] = self.train_df['class'].map(self.class_mapper)

            print(f"Created mappings for {self.num_classes} classes")

        except Exception as e:
            print(f"Error creating class mappings: {e}")
            raise

    def _analyze_dataset(self):
        """
        Analyze the dataset and compute statistics
        """
        print("\nAnalyzing dataset...")

        # Class distribution
        self.class_distribution = self.train_df['class'].value_counts()

        # Image dimensions
        print("Calculating image dimensions...")
        for img_path in tqdm(self.train_df['image_path'].unique()):
            try:
                img = cv2.imread(str(img_path))
                if img is not None:
                    self.image_dimensions.append(img.shape[:2])
            except Exception as e:
                print(f"Warning: Could not read image {img_path}: {e}")

    def print_dataset_info(self):
        """
        Print dataset statistics and information
        """
        print("\n=== Dataset Information ===")
        print(f"Total training images: {len(self.train_df['Image_ID'].unique())}")
        print(f"Total test images: {len(self.test_df)}")
        print(f"Number of classes: {self.num_classes}")

        print("\nClass Distribution:")
        for class_name, count in self.class_distribution.items():
            print(f"{class_name}: {count} ({count/len(self.train_df)*100:.2f}%)")

        if self.image_dimensions:
            heights, widths = zip(*self.image_dimensions)
            print(f"\nImage Dimensions:")
            print(f"Height - Min: {min(heights)}, Max: {max(heights)}, Mean: {np.mean(heights):.0f}")
            print(f"Width  - Min: {min(widths)}, Max: {max(widths)}, Mean: {np.mean(widths):.0f}")

    def prepare_yolo_dataset(self):
        """
        Prepare the dataset in YOLO format
        """
        try:
            print("\nPreparing YOLO dataset...")

            # Split data into train and validation sets
            train_imgs, val_imgs = self._split_dataset()

            # Process training set
            self._process_split(train_imgs, 'train')

            # Process validation set
            self._process_split(val_imgs, 'val')

            # Process test set
            self._process_test_set()

            # Create YAML file
            yaml_path = self._create_yaml_file()

            print("YOLO dataset preparation completed!")
            return yaml_path

        except Exception as e:
            print(f"Error preparing YOLO dataset: {e}")
            raise

    def _split_dataset(self):
        """Split dataset into training and validation sets"""
        unique_images = self.train_df.drop_duplicates(subset=['Image_ID'])
        return train_test_split(
            unique_images,
            test_size=0.25,
            random_state=42,
            stratify=unique_images['class']
        )

    def _process_split(self, image_group, split_name):
        """Process a data split (train/val)"""
        print(f"\nProcessing {split_name} split...")

        img_dir = self.dataset_path / 'images' / split_name
        label_dir = self.dataset_path / 'labels' / split_name

        for _, row in tqdm(image_group.iterrows(), desc=f"Processing {split_name} images"):
            # Copy image
            src_path = row['image_path']
            dst_path = img_dir / src_path.name
            shutil.copy(str(src_path), str(dst_path))

            # Create YOLO format labels
            self._create_yolo_label(row, label_dir)

    def _create_yolo_label(self, row, label_dir):
        """Create YOLO format label file for an image"""
        img_path = row['image_path']
        img = cv2.imread(str(img_path))
        if img is None:
            print(f"Warning: Could not read image {img_path}")
            return

        height, width = img.shape[:2]
        annotations = self.train_df[self.train_df.Image_ID == row.Image_ID]

        label_path = label_dir / f"{img_path.stem}.txt"
        with open(label_path, 'w') as f:
            for _, ann in annotations.iterrows():
                # Convert to YOLO format
                x_center = ((ann['xmin'] + ann['xmax']) / 2) / width
                y_center = ((ann['ymin'] + ann['ymax']) / 2) / height
                box_width = (ann['xmax'] - ann['xmin']) / width
                box_height = (ann['ymax'] - ann['ymin']) / height

                f.write(f"{ann['class_id']} {x_center:.6f} {y_center:.6f} {box_width:.6f} {box_height:.6f}\n")

    def _process_test_set(self):
        """Process test set images"""
        print("\nProcessing test set...")
        test_dir = self.dataset_path / 'images' / 'test'

        for _, row in tqdm(self.test_df.iterrows(), desc="Copying test images"):
            src_path = row['image_path']
            dst_path = test_dir / src_path.name
            shutil.copy(str(src_path), str(dst_path))

    def _create_yaml_file(self):
        """Create YAML configuration file for YOLO"""
        yaml_path = 'data.yaml'
        yaml_content = {
            'path': str(self.dataset_path),
            'train': str(self.dataset_path / 'images/train'),
            'val': str(self.dataset_path / 'images/val'),
            'test': str(self.dataset_path / 'images/test'),
            'nc': self.num_classes,
            'names': list(self.class_mapper.keys())
        }

        with open(yaml_path, 'w') as f:
            yaml.dump(yaml_content, f, default_flow_style=False)

        print(f"\nCreated YAML configuration file: {yaml_path}")
        print("\nYAML contents:")
        !cat {yaml_path}

        return yaml_path

In [ ]:
import os
import shutil
from pathlib import Path
import pandas as pd
import yaml
import cv2
from tqdm import tqdm
from sklearn.model_selection import train_test_split

def verify_and_fix_paths():
    """Verify and fix all necessary paths."""
    print("Verifying paths and files...")

    # Get current working directory
    cwd = os.getcwd()
    print(f"Current working directory: {cwd}")

    # List all files in the current directory
    print("\nFiles in current directory:")
    !ls -la

    # Create dataset directories
    dataset_path = Path('/content/dataset')
    images_path = dataset_path / 'images'
    labels_path = dataset_path / 'labels'

    # Create directories if they don't exist
    for p in [dataset_path, images_path, labels_path]:
        p.mkdir(parents=True, exist_ok=True)

    return dataset_path, images_path, labels_path

class DataPreprocessor:
    def __init__(self, base_path: Path):
        self.base_path = base_path
        self.dataset_path = Path('/content/dataset')
        self.class_mapper = {}  # Initialize class mapper

    def create_class_mappings(self):
        """Create a mapping from class names to numerical IDs."""
        # Get the unique class names from the dataset
        class_names = self.train_df['class'].unique()

        # Create a mapping from class names to numeric IDs
        self.class_mapper = {name: idx for idx, name in enumerate(class_names)}
        print(f"Class mappings created: {self.class_mapper}")

    def load_data(self):
        """Load and prepare training and testing data."""
        try:
            # Load CSV files with explicit path
            print(f"Loading data from {self.base_path}")
            self.train_df = pd.read_csv(self.base_path / 'Train.csv')
            self.test_df = pd.read_csv(self.base_path / 'Test.csv')

            # Unzip images if needed
            images_zip_path = self.base_path / 'images.zip'
            if images_zip_path.exists():
                print(f"Unzipping images from {images_zip_path}")
                !unzip -q "{str(images_zip_path)}" -d "/content/images"

            # Add image paths
            self.train_df['image_path'] = [
                Path('/content/images/' + x) for x in self.train_df.Image_ID
            ]
            self.test_df['image_path'] = [
                Path('/content/images/' + x) for x in self.test_df.Image_ID
            ]

            # Create class mappings
            self.create_class_mappings()
            print("Data loaded successfully!")

        except Exception as e:
            print(f"Error loading data: {e}")
            raise

    def prepare_yolo_dataset(self):
        """Prepare dataset in YOLO format."""
        try:
            print("Preparing YOLO dataset...")

            # Create necessary directories
            train_dir = self.dataset_path / 'images/train'
            val_dir = self.dataset_path / 'images/val'
            test_dir = self.dataset_path / 'images/test'

            train_labels_dir = self.dataset_path / 'labels/train'
            val_labels_dir = self.dataset_path / 'labels/val'
            test_labels_dir = self.dataset_path / 'labels/test'

            # Create all directories
            for d in [train_dir, val_dir, test_dir, train_labels_dir, val_labels_dir, test_labels_dir]:
                d.mkdir(parents=True, exist_ok=True)

            # Split data into train and validation sets
            unique_images = self.train_df.drop_duplicates(subset=['Image_ID'])
            train_imgs, val_imgs = train_test_split(
                unique_images,
                test_size=0.25,
                random_state=42,
                stratify=unique_images['class']
            )

            # Process train and validation sets
            def process_dataset(image_group, img_dir, label_dir, set_name):
                print(f"\nProcessing {set_name} set...")
                for _, row in tqdm(image_group.iterrows()):
                    # Copy image
                    src_path = row['image_path']
                    dst_path = img_dir / src_path.name
                    shutil.copy(str(src_path), str(dst_path))

                    # Create labels
                    image_annotations = self.train_df[self.train_df.Image_ID == row.Image_ID]
                    label_path = label_dir / f"{src_path.stem}.txt"

                    # Get image dimensions
                    img = cv2.imread(str(src_path))
                    if img is None:
                        print(f"Warning: Could not read image {src_path}")
                        continue
                    height, width = img.shape[:2]

                    with open(label_path, 'w') as f:
                        for _, ann in image_annotations.iterrows():
                            # Convert to YOLO format
                            x_center = ((ann['xmin'] + ann['xmax']) / 2) / width
                            y_center = ((ann['ymin'] + ann['ymax']) / 2) / height
                            box_width = (ann['xmax'] - ann['xmin']) / width
                            box_height = (ann['ymax'] - ann['ymin']) / height
                            class_id = self.class_mapper[ann['class']]

                            # Write YOLO format line
                            f.write(f"{class_id} {x_center} {y_center} {box_width} {box_height}\n")

            # Process datasets
            process_dataset(train_imgs, train_dir, train_labels_dir, "training")
            process_dataset(val_imgs, val_dir, val_labels_dir, "validation")

            # Copy test images
            print("\nProcessing test set...")
            for _, row in tqdm(self.test_df.iterrows()):
                src_path = row['image_path']
                dst_path = test_dir / src_path.name
                shutil.copy(str(src_path), str(dst_path))

            # Create data.yaml file
            print("\nCreating data.yaml file...")
            data_yaml = {
                'path': str(self.dataset_path),  # Base path
                'train': str(train_dir),  # Train images path
                'val': str(val_dir),      # Validation images path
                'test': str(test_dir),    # Test images path
                'nc': len(self.class_mapper),  # Number of classes
                'names': list(self.class_mapper.keys())  # Class names
            }

            yaml_path = 'data.yaml'
            with open(yaml_path, 'w') as f:
                yaml.dump(data_yaml, f, default_flow_style=False)

            print(f"Dataset prepared successfully! YAML file created at: {yaml_path}")
            print("\nYAML contents:")
            !cat data.yaml

            return yaml_path

        except Exception as e:
            print(f"Error preparing YOLO dataset: {e}")
            raise

def main():
    try:
        # Initialize configuration
        config = Config()  # Ensure Config is defined elsewhere

        # Verify and fix paths
        dataset_path, images_path, labels_path = verify_and_fix_paths()

        # Initialize data preprocessor
        print("\nInitializing data preprocessor...")
        preprocessor = DataPreprocessor(Path(BASE_PATH))  # Ensure BASE_PATH is set elsewhere

        # Load data
        print("\nLoading data...")
        preprocessor.load_data()

        # Prepare YOLO dataset
        print("\nPreparing YOLO dataset...")
        yaml_path = preprocessor.prepare_yolo_dataset()

        # Initialize model training
        print("\nInitializing model training...")
        trainer = ModelTrainer(config)  # Ensure ModelTrainer is defined elsewhere
        training_results = trainer.train(yaml_path)

        return preprocessor, trainer, training_results

    except Exception as e:
        print(f"\nError in main execution: {e}")
        raise

# Execute the pipeline
if __name__ == "__main__":
    preprocessor, trainer, results = main()



Error in main execution: name 'Config' is not defined


NameError: name 'Config' is not defined

In [ ]:
# Import Necessary Libraries
import os
import shutil
from pathlib import Path
import pandas as pd
import yaml
import cv2
from tqdm import tqdm
from sklearn.model_selection import train_test_split

# Ensure Ultralytics YOLO is installed
try:
    from ultralytics import YOLO
except ImportError:
    print("Ultralytics YOLO not found. Installing...")
    !pip install ultralytics
    from ultralytics import YOLO

# Define BASE_PATH
BASE_PATH = '/content/drive/MyDrive/ghana-crop-disease-detection-challenge20241003-7123-59ht2i'  # Update this path as needed

def verify_and_fix_paths():
    """Verify and fix all necessary paths."""
    print("Verifying paths and files...")

    # Get current working directory
    cwd = os.getcwd()
    print(f"Current working directory: {cwd}")

    # List all files in the current directory
    print("\nFiles in current directory:")
    !ls -la

    # Create dataset directories
    dataset_path = Path('/content/dataset')
    images_path = dataset_path / 'images'
    labels_path = dataset_path / 'labels'

    # Create directories if they don't exist
    for p in [dataset_path, images_path, labels_path]:
        p.mkdir(parents=True, exist_ok=True)

    return dataset_path, images_path, labels_path

class Config:
    """Configuration settings for data preprocessing and model training."""
    def __init__(self):
        # Data Preprocessing Configurations
        self.base_path = Path(BASE_PATH)
        self.dataset_path = Path('/content/dataset')
        self.images_dir = self.dataset_path / 'images'
        self.labels_dir = self.dataset_path / 'labels'

        # YOLO Model Training Configurations
        self.model_weights = 'yolov8n.pt'  # Pretrained YOLOv8n model
        self.img_size = 640
        self.batch_size = 16
        self.epochs = 50
        self.learning_rate = 0.01
        self.data_yaml = 'data.yaml'  # Path to data.yaml file
        self.project = 'runs/train'  # Directory to save training runs
        self.name = 'yolov8n_custom'  # Name of the training run
        self.device = '0'  # GPU device, set to 'cpu' if GPU not available

class DataPreprocessor:
    def __init__(self, config: Config):
        self.config = config
        self.base_path = self.config.base_path
        self.dataset_path = self.config.dataset_path
        self.images_path = self.config.images_dir
        self.labels_path = self.config.labels_dir
        self.class_mapper = {}  # Initialize class mapper

    def create_class_mappings(self):
        """Create a mapping from class names to numerical IDs."""
        # Get the unique class names from the dataset
        class_names = self.train_df['class'].unique()

        # Create a mapping from class names to numeric IDs
        self.class_mapper = {name: idx for idx, name in enumerate(sorted(class_names))}
        print(f"Class mappings created: {self.class_mapper}")

    def load_data(self):
        """Load and prepare training and testing data."""
        try:
            # Load CSV files with explicit path
            print(f"Loading data from {self.base_path}")
            self.train_df = pd.read_csv(self.base_path / 'Train.csv')
            self.test_df = pd.read_csv(self.base_path / 'Test.csv')

            # Unzip images if needed
            images_zip_path = self.base_path / 'images.zip'
            if images_zip_path.exists():
                print(f"Unzipping images from {images_zip_path}")
                !unzip -o -q "{str(images_zip_path)}" -d "/content/images"
            else:
                print(f"No images.zip found at {images_zip_path}. Assuming images are already extracted.")

            # Add image paths
            self.train_df['image_path'] = self.train_df['Image_ID'].apply(lambda x: Path('/content/images') / x)
            self.test_df['image_path'] = self.test_df['Image_ID'].apply(lambda x: Path('/content/images') / x)

            # Create class mappings
            self.create_class_mappings()
            print("Data loaded successfully!")

        except Exception as e:
            print(f"Error loading data: {e}")
            raise

    def prepare_yolo_dataset(self):
        """Prepare dataset in YOLO format."""
        try:
            print("Preparing YOLO dataset...")

            # Create necessary directories
            train_dir = self.dataset_path / 'images/train'
            val_dir = self.dataset_path / 'images/val'
            test_dir = self.dataset_path / 'images/test'

            train_labels_dir = self.dataset_path / 'labels/train'
            val_labels_dir = self.dataset_path / 'labels/val'
            test_labels_dir = self.dataset_path / 'labels/test'

            # Create all directories
            for d in [train_dir, val_dir, test_dir, train_labels_dir, val_labels_dir, test_labels_dir]:
                d.mkdir(parents=True, exist_ok=True)

            # Split data into train and validation sets
            unique_images = self.train_df.drop_duplicates(subset=['Image_ID'])
            train_imgs, val_imgs = train_test_split(
                unique_images,
                test_size=0.25,
                random_state=42,
                stratify=unique_images['class']
            )

            # Process train and validation sets
            def process_dataset(image_group, img_dir, label_dir, set_name):
                print(f"\nProcessing {set_name} set...")
                for _, row in tqdm(image_group.iterrows(), total=image_group.shape[0]):
                    # Copy image
                    src_path = row['image_path']
                    dst_path = img_dir / src_path.name
                    if not dst_path.exists():
                        shutil.copy(str(src_path), str(dst_path))

                    # Create labels
                    image_annotations = self.train_df[self.train_df.Image_ID == row.Image_ID]
                    label_path = label_dir / f"{src_path.stem}.txt"

                    # Get image dimensions
                    img = cv2.imread(str(src_path))
                    if img is None:
                        print(f"Warning: Could not read image {src_path}")
                        continue
                    height, width = img.shape[:2]

                    with open(label_path, 'w') as f:
                        for _, ann in image_annotations.iterrows():
                            # Convert to YOLO format
                            x_center = ((ann['xmin'] + ann['xmax']) / 2) / width
                            y_center = ((ann['ymin'] + ann['ymax']) / 2) / height
                            box_width = (ann['xmax'] - ann['xmin']) / width
                            box_height = (ann['ymax'] - ann['ymin']) / height
                            class_id = self.class_mapper.get(ann['class'])

                            if class_id is None:
                                print(f"Warning: Class '{ann['class']}' not found in class_mapper.")
                                continue

                            # Write YOLO format line
                            f.write(f"{class_id} {x_center:.6f} {y_center:.6f} {box_width:.6f} {box_height:.6f}\n")

            # Process datasets
            process_dataset(train_imgs, train_dir, train_labels_dir, "training")
            process_dataset(val_imgs, val_dir, val_labels_dir, "validation")

            # Copy test images
            print("\nProcessing test set...")
            for _, row in tqdm(self.test_df.iterrows(), total=self.test_df.shape[0]):
                src_path = row['image_path']
                dst_path = test_dir / src_path.name
                if not dst_path.exists():
                    shutil.copy(str(src_path), str(dst_path))

            # Create data.yaml file
            print("\nCreating data.yaml file...")
            data_yaml = {
                'path': str(self.dataset_path.resolve()),  # Base path
                'train': 'images/train',  # Relative path from 'path'
                'val': 'images/val',      # Relative path from 'path'
                'test': 'images/test',    # Relative path from 'path'
                'nc': len(self.class_mapper),  # Number of classes
                'names': list(self.class_mapper.keys())  # Class names
            }

            yaml_path = self.dataset_path / self.config.data_yaml
            with open(yaml_path, 'w') as f:
                yaml.dump(data_yaml, f, default_flow_style=False)

            print(f"Dataset prepared successfully! YAML file created at: {yaml_path}")
            print("\nYAML contents:")
            print(yaml.dump(data_yaml, default_flow_style=False))

            return yaml_path

        except Exception as e:
            print(f"Error preparing YOLO dataset: {e}")
            raise

class ModelTrainer:
    def __init__(self, config: Config):
        self.config = config
        self.model = None  # Placeholder for the YOLO model

    def train(self, yaml_path: Path):
        """Train the YOLOv8 model using the prepared dataset."""
        try:
            print("Initializing YOLOv8 model for training...")
            self.model = YOLO(self.config.model_weights)

            print("Starting training...")
            results = self.model.train(
                data=str(yaml_path),
                imgsz=self.config.img_size,
                batch=self.config.batch_size,
                epochs=self.config.epochs,
                lr0=self.config.learning_rate,
                project=self.config.project,
                name=self.config.name,
                device=self.config.device,
                verbose=True
            )
            print("Training completed successfully!")
            return results
        except Exception as e:
            print(f"Error during model training: {e}")
            raise

def main():
    try:
        # Initialize configuration
        config = Config()

        # Verify and fix paths
        dataset_path, images_path, labels_path = verify_and_fix_paths()

        # Initialize data preprocessor
        print("\nInitializing data preprocessor...")
        preprocessor = DataPreprocessor(config)

        # Load data
        print("\nLoading data...")
        preprocessor.load_data()

        # Prepare YOLO dataset
        print("\nPreparing YOLO dataset...")
        yaml_path = preprocessor.prepare_yolo_dataset()

        # Initialize model training
        print("\nInitializing model training...")
        trainer = ModelTrainer(config)
        training_results = trainer.train(yaml_path)

        return preprocessor, trainer, training_results

    except Exception as e:
        print(f"\nError in main execution: {e}")
        raise

# Execute the pipeline
if __name__ == "__main__":
    preprocessor, trainer, results = main()


Verifying paths and files...
Current working directory: /content/drive/MyDrive/ghana-crop-disease-detection-challenge20241003-7123-59ht2i

Files in current directory:
total 10885781
drwx------ 2 root root        4096 Oct 15 20:33 dataset
-rw------- 1 root root         602 Oct 16 21:21 data.yaml
drwx------ 2 root root        4096 Oct 16 20:02 ghana-crop-disease-detection-challenge20241003-7123-59ht2i
-rw------- 1 root root 11127738191 Oct 12 15:33 images.zip
-rw------- 1 root root        1091 Oct 15 09:01 manifest-8517f262e2ca61a7f7ac8fc1d32eb0be20241003-7123-1lvxb66.json
-rw------- 1 root root     5763966 Oct 16 19:57 Rail_Challenge_Starter.ipynb
drwx------ 3 root root        4096 Oct 16 20:24 runs
-rw------- 1 root root     2512842 Oct 15 09:01 SampleSubmission.csv
-rw------- 1 root root       42066 Oct 15 09:01 Test.csv
-rw------- 1 root root     4416736 Oct 15 09:01 Train.csv
-rw------- 1 root root     6549796 Oct 16 20:24 yolov8n.pt

Initializing data preprocessor...

Loading data.

100%|██████████| 3676/3676 [07:36<00:00,  8.05it/s]



Processing validation set...


100%|██████████| 1226/1226 [02:25<00:00,  8.41it/s]



Processing test set...


100%|██████████| 2101/2101 [00:38<00:00, 54.80it/s]



Creating data.yaml file...
Dataset prepared successfully! YAML file created at: /content/dataset/data.yaml

YAML contents:
names:
- Corn_Cercospora_Leaf_Spot
- Corn_Common_Rust
- Corn_Healthy
- Corn_Northern_Leaf_Blight
- Corn_Streak
- Pepper_Bacterial_Spot
- Pepper_Cercospora
- Pepper_Early_Blight
- Pepper_Fusarium
- Pepper_Healthy
- Pepper_Late_Blight
- Pepper_Leaf_Blight
- Pepper_Leaf_Curl
- Pepper_Leaf_Mosaic
- Pepper_Septoria
- Tomato_Bacterial_Spot
- Tomato_Early_Blight
- Tomato_Fusarium
- Tomato_Healthy
- Tomato_Late_Blight
- Tomato_Leaf_Curl
- Tomato_Mosaic
- Tomato_Septoria
nc: 23
path: /content/dataset
test: images/test
train: images/train
val: images/val


Initializing model training...
Initializing YOLOv8 model for training...
Starting training...
Ultralytics 8.3.15 🚀 Python-3.10.12 torch-2.4.1+cu121 
Error during model training: Invalid CUDA 'device=0' requested. Use 'device=cpu' or pass valid CUDA device(s) if available, i.e. 'device=0' or 'device=0,1,2,3' for Multi-GPU.

ValueError: Invalid CUDA 'device=0' requested. Use 'device=cpu' or pass valid CUDA device(s) if available, i.e. 'device=0' or 'device=0,1,2,3' for Multi-GPU.

torch.cuda.is_available(): False
torch.cuda.device_count(): 0
os.environ['CUDA_VISIBLE_DEVICES']: None
See https://pytorch.org/get-started/locally/ for up-to-date torch install instructions if no CUDA devices are seen by torch.


In [ ]:
# Import Necessary Libraries
import os
import shutil
from pathlib import Path
import pandas as pd
import yaml
import cv2
from tqdm import tqdm
from sklearn.model_selection import train_test_split

# Ensure Ultralytics YOLO is installed
try:
    from ultralytics import YOLO
except ImportError:
    print("Ultralytics YOLO not found. Installing...")
    !pip install ultralytics
    from ultralytics import YOLO

# Define BASE_PATH
BASE_PATH = '/content/drive/MyDrive/ghana-crop-disease-detection-challenge20241003-7123-59ht2i'  # Update this path as needed

def verify_and_fix_paths():
    """Verify and fix all necessary paths."""
    print("Verifying paths and files...")

    # Get current working directory
    cwd = os.getcwd()
    print(f"Current working directory: {cwd}")

    # List all files in the current directory
    print("\nFiles in current directory:")
    !ls -la

    # Create dataset directories
    dataset_path = Path('/content/dataset')
    images_path = dataset_path / 'images'
    labels_path = dataset_path / 'labels'

    # Create directories if they don't exist
    for p in [dataset_path, images_path, labels_path]:
        p.mkdir(parents=True, exist_ok=True)

    return dataset_path, images_path, labels_path

class Config:
    """Configuration settings for data preprocessing and model training."""
    def __init__(self):
        # Data Preprocessing Configurations
        self.base_path = Path(BASE_PATH)
        self.dataset_path = Path('/content/dataset')
        self.images_dir = self.dataset_path / 'images'
        self.labels_dir = self.dataset_path / 'labels'

        # YOLO Model Training Configurations
        self.model_weights = 'yolov8n.pt'  # Pretrained YOLOv8n model
        self.img_size = 640
        self.batch_size = 16
        self.epochs = 5
        self.learning_rate = 0.01
        self.data_yaml = 'data.yaml'  # Path to data.yaml file
        self.project = 'runs/train'  # Directory to save training runs
        self.name = 'yolov8n_custom'  # Name of the training run
        self.device = 'cpu'  # Set to 'cpu' since CUDA device is unavailable

class DataPreprocessor:
    def __init__(self, config: Config):
        self.config = config
        self.base_path = self.config.base_path
        self.dataset_path = self.config.dataset_path
        self.images_path = self.config.images_dir
        self.labels_path = self.config.labels_dir
        self.class_mapper = {}  # Initialize class mapper

    def create_class_mappings(self):
        """Create a mapping from class names to numerical IDs."""
        # Get the unique class names from the dataset
        class_names = self.train_df['class'].unique()

        # Create a mapping from class names to numeric IDs
        self.class_mapper = {name: idx for idx, name in enumerate(sorted(class_names))}
        print(f"Class mappings created: {self.class_mapper}")

    def load_data(self):
        """Load and prepare training and testing data."""
        try:
            # Load CSV files with explicit path
            print(f"Loading data from {self.base_path}")
            self.train_df = pd.read_csv(self.base_path / 'Train.csv')
            self.test_df = pd.read_csv(self.base_path / 'Test.csv')

            # Unzip images if needed
            images_zip_path = self.base_path / 'images.zip'
            if images_zip_path.exists():
                print(f"Unzipping images from {images_zip_path}")
                !unzip -o -q "{str(images_zip_path)}" -d "/content/images"
            else:
                print(f"No images.zip found at {images_zip_path}. Assuming images are already extracted.")

            # Add image paths
            self.train_df['image_path'] = self.train_df['Image_ID'].apply(lambda x: Path('/content/images') / x)
            self.test_df['image_path'] = self.test_df['Image_ID'].apply(lambda x: Path('/content/images') / x)

            # Create class mappings
            self.create_class_mappings()
            print("Data loaded successfully!")

        except Exception as e:
            print(f"Error loading data: {e}")
            raise

    def prepare_yolo_dataset(self):
        """Prepare dataset in YOLO format."""
        try:
            print("Preparing YOLO dataset...")

            # Create necessary directories
            train_dir = self.dataset_path / 'images/train'
            val_dir = self.dataset_path / 'images/val'
            test_dir = self.dataset_path / 'images/test'

            train_labels_dir = self.dataset_path / 'labels/train'
            val_labels_dir = self.dataset_path / 'labels/val'
            test_labels_dir = self.dataset_path / 'labels/test'

            # Create all directories
            for d in [train_dir, val_dir, test_dir, train_labels_dir, val_labels_dir, test_labels_dir]:
                d.mkdir(parents=True, exist_ok=True)

            # Split data into train and validation sets
            unique_images = self.train_df.drop_duplicates(subset=['Image_ID'])
            train_imgs, val_imgs = train_test_split(
                unique_images,
                test_size=0.25,
                random_state=42,
                stratify=unique_images['class']
            )

            # Process train and validation sets
            def process_dataset(image_group, img_dir, label_dir, set_name):
                print(f"\nProcessing {set_name} set...")
                for _, row in tqdm(image_group.iterrows(), total=image_group.shape[0]):
                    # Copy image
                    src_path = row['image_path']
                    dst_path = img_dir / src_path.name
                    if not dst_path.exists():
                        shutil.copy(str(src_path), str(dst_path))

                    # Create labels
                    image_annotations = self.train_df[self.train_df.Image_ID == row.Image_ID]
                    label_path = label_dir / f"{src_path.stem}.txt"

                    # Get image dimensions
                    img = cv2.imread(str(src_path))
                    if img is None:
                        print(f"Warning: Could not read image {src_path}")
                        continue
                    height, width = img.shape[:2]

                    with open(label_path, 'w') as f:
                        for _, ann in image_annotations.iterrows():
                            # Convert to YOLO format
                            x_center = ((ann['xmin'] + ann['xmax']) / 2) / width
                            y_center = ((ann['ymin'] + ann['ymax']) / 2) / height
                            box_width = (ann['xmax'] - ann['xmin']) / width
                            box_height = (ann['ymax'] - ann['ymin']) / height
                            class_id = self.class_mapper.get(ann['class'])

                            if class_id is None:
                                print(f"Warning: Class '{ann['class']}' not found in class_mapper.")
                                continue

                            # Write YOLO format line
                            f.write(f"{class_id} {x_center:.6f} {y_center:.6f} {box_width:.6f} {box_height:.6f}\n")

            # Process datasets
            process_dataset(train_imgs, train_dir, train_labels_dir, "training")
            process_dataset(val_imgs, val_dir, val_labels_dir, "validation")

            # Copy test images
            print("\nProcessing test set...")
            for _, row in tqdm(self.test_df.iterrows(), total=self.test_df.shape[0]):
                src_path = row['image_path']
                dst_path = test_dir / src_path.name
                if not dst_path.exists():
                    shutil.copy(str(src_path), str(dst_path))

            # Create data.yaml file
            print("\nCreating data.yaml file...")
            data_yaml = {
                'path': str(self.dataset_path.resolve()),  # Base path
                'train': 'images/train',  # Relative path from 'path'
                'val': 'images/val',      # Relative path from 'path'
                'test': 'images/test',    # Relative path from 'path'
                'nc': len(self.class_mapper),  # Number of classes
                'names': list(self.class_mapper.keys())  # Class names
            }

            yaml_path = self.dataset_path / self.config.data_yaml
            with open(yaml_path, 'w') as f:
                yaml.dump(data_yaml, f, default_flow_style=False)

            print(f"Dataset prepared successfully! YAML file created at: {yaml_path}")
            print("\nYAML contents:")
            print(yaml.dump(data_yaml, default_flow_style=False))

            return yaml_path

        except Exception as e:
            print(f"Error preparing YOLO dataset: {e}")
            raise

class ModelTrainer:
    def __init__(self, config: Config):
        self.config = config
        self.model = None  # Placeholder for the YOLO model

    def train(self, yaml_path: Path):
        """Train the YOLOv8 model using the prepared dataset."""
        try:
            print("Initializing YOLOv8 model for training...")
            self.model = YOLO(self.config.model_weights)

            print("Starting training...")
            results = self.model.train(
                data=str(yaml_path),
                imgsz=self.config.img_size,
                batch=self.config.batch_size,
                epochs=self.config.epochs,
                lr0=self.config.learning_rate,
                project=self.config.project,
                name=self.config.name,
                device=self.config.device
            )
            print("Training complete!")
            return results

        except Exception as e:
            print(f"Error during model training: {e}")
            raise

# Main Execution
if __name__ == '__main__':
    try:
        # Step 1: Create Config Object
        config = Config()

        # Step 2: Prepare Dataset
        preprocessor = DataPreprocessor(config)
        preprocessor.load_data()
        yaml_path = preprocessor.prepare_yolo_dataset()

        # Step 3: Train Model
        trainer = ModelTrainer(config)
        trainer.train(yaml_path)

    except Exception as e:
        print(f"Error in main execution: {e}")


Loading data from /content/drive/MyDrive/ghana-crop-disease-detection-challenge20241003-7123-59ht2i
Unzipping images from /content/drive/MyDrive/ghana-crop-disease-detection-challenge20241003-7123-59ht2i/images.zip
Class mappings created: {'Corn_Cercospora_Leaf_Spot': 0, 'Corn_Common_Rust': 1, 'Corn_Healthy': 2, 'Corn_Northern_Leaf_Blight': 3, 'Corn_Streak': 4, 'Pepper_Bacterial_Spot': 5, 'Pepper_Cercospora': 6, 'Pepper_Early_Blight': 7, 'Pepper_Fusarium': 8, 'Pepper_Healthy': 9, 'Pepper_Late_Blight': 10, 'Pepper_Leaf_Blight': 11, 'Pepper_Leaf_Curl': 12, 'Pepper_Leaf_Mosaic': 13, 'Pepper_Septoria': 14, 'Tomato_Bacterial_Spot': 15, 'Tomato_Early_Blight': 16, 'Tomato_Fusarium': 17, 'Tomato_Healthy': 18, 'Tomato_Late_Blight': 19, 'Tomato_Leaf_Curl': 20, 'Tomato_Mosaic': 21, 'Tomato_Septoria': 22}
Data loaded successfully!
Preparing YOLO dataset...

Processing training set...


100%|██████████| 3676/3676 [04:47<00:00, 12.78it/s]



Processing validation set...


100%|██████████| 1226/1226 [01:34<00:00, 12.98it/s]



Processing test set...


...

                   all       1226      10252      0.306      0.178     0.0977      0.039

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        5/5         0G      2.282      2.857       1.51        132        640: 100%|██████████| 230/230 [55:57<00:00, 14.60s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 39/39 [06:24<00:00,  9.86s/it]


                   all       1226      10252      0.284      0.183      0.115     0.0467

5 epochs completed in 5.348 hours.
Optimizer stripped from runs/train/yolov8n_custom3/weights/last.pt, 5.6MB
Optimizer stripped from runs/train/yolov8n_custom3/weights/best.pt, 5.6MB

Validating runs/train/yolov8n_custom3/weights/best.pt...
Ultralytics 8.3.15 🚀 Python-3.10.12 torch-2.4.1+cu121 CPU (Intel Xeon 2.20GHz)
Model summary (fused): 186 layers, 2,688,853 parameters, 0 gradients, 6.8 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 39/39 [05:13<00:00,  8.04s/it]


                   all       1226      10252      0.328      0.183      0.116     0.0467
Corn_Cercospora_Leaf_Spot        202       1600      0.326      0.409      0.281      0.103
      Corn_Common_Rust         79        442      0.265       0.36       0.21     0.0815
          Corn_Healthy         63        415      0.205      0.354      0.165     0.0572
Corn_Northern_Leaf_Blight         12         25          0          0          0          0
           Corn_Streak         87        804      0.215      0.541      0.244     0.0763
 Pepper_Bacterial_Spot        132        476      0.205      0.144     0.0887     0.0214
     Pepper_Cercospora         41        121          1          0      0.025    0.00723
   Pepper_Early_Blight          2         15          0          0          0          0
       Pepper_Fusarium         52        123      0.222      0.179      0.111     0.0362
        Pepper_Healthy         68        184      0.316       0.63      0.416      0.176
    Pepper_Late

In [10]:
from ultralytics import YOLO

# Load a model
model = YOLO('yolov8n.pt')  # Make sure this path is correct
print("Model loaded successfully!")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Model loaded successfully!
